In [1]:
import $file.common
import common._
import doobie._, doobie.implicits._
import cats._, cats.implicits._
import cats.effect._
import fs2.Stream
import java.sql._
import doobie.hikari._

import $file.$     

import common._

import doobie._, doobie.implicits._

import cats._, cats.implicits._

import cats.effect._

import fs2.Stream

import java.sql._

import doobie.hikari._

# Variation 5. DAO in Scala

Common case classes for DAO interfaces.

In [2]:
case class Country(code: String, name: String, capital: Option[Int])
case class City(id: Int, name: String, countryCode: String, population: Int)

defined class Country
defined class City

# Id

In [3]:
trait CityDAOId{
    def city(id: Int): City
    def cityName(id: Int): String
    def cityPopulation(id: Int): Int
    def cityCountryCode(id: Int): String
}

trait CountryDAOId{
    def country(code: String): Country
    def countryName(code: String): String
    def countryCapital(code: String): Option[Int]
}

trait WorldId extends CityDAOId with CountryDAOId{
    def countries(): List[String]
    def cities(): List[Int]
}

defined trait CityDAOId
defined trait CountryDAOId
defined trait WorldId

In [4]:
def largeCitiesId(W: WorldId): List[(String, String)] = 
    for {
        countryCode <- W.countries
        Country(_, name, Some(capitalId)) <- List(W.country(countryCode))
        city <- List(W.city(capitalId)) if city.population > 8000000
    } yield (city.name, name)

defined function largeCitiesId

In [5]:
def largeCitiesId2(W: WorldId): List[(String, String)] = 
    for {
        countryCode <- W.countries
        country = W.country(countryCode) if country.capital.isDefined
        city = W.city(country.capital.get) if city.population > 8000000
    } yield (city.name, country.name)

defined function largeCitiesId2

In [6]:
class DoobieWorldId(xa: Transactor[IO]) extends WorldId{
    def city(id: Int): City = 
        sql"select id, name, countryCode, population from city where id = $id"
            .query[City].unique.transact(xa).unsafeRunSync

    def cityName(id: Int): String = 
        sql"select name from city where id = $id"
            .query[String].unique.transact(xa).unsafeRunSync
    
    def cityPopulation(id: Int): Int = 
        sql"select population from city where id = $id"
            .query[Int].unique.transact(xa).unsafeRunSync
    
    def cityCountryCode(id: Int): String = 
        sql"select countryCode from city where id = $id"
            .query[String].unique.transact(xa).unsafeRunSync

    def country(code: String): Country = 
        sql"select code, name, capital from country where code = $code"
            .query[Country].unique.transact(xa).unsafeRunSync
    
    def countryName(code: String): String =
        sql"select name from country where code = $code"
            .query[String].unique.transact(xa).unsafeRunSync
    
    def countryCapital(code: String): Option[Int] =
        sql"select capital from country where code = $code"
            .query[Int].option.transact(xa).unsafeRunSync
    
    def countries(): List[String] = 
        sql"select code from country"
            .query[String].to[List].transact(xa).unsafeRunSync
    
    def cities(): List[Int] = 
        sql"select id from city"
            .query[Int].to[List].transact(xa).unsafeRunSync
}

object DoobieWorldId extends DoobieWorldId(xa)

defined class DoobieWorldId
defined object DoobieWorldId

In [7]:
largeCitiesId2(DoobieWorldId).timed

res6: (List[(String, String)], Long) = (
  List(
    ("Jakarta", "Indonesia"),
    ("Seoul", "South Korea"),
    ("Ciudad de M\u00e9xico", "Mexico"),
    ("Moscow", "Russian Federation")
  ),
  4838271751L
)

### IO

In [8]:
trait CityDAO{
    def city(id: Int): IO[City]
    def cityName(id: Int): IO[String]
    def cityPopulation(id: Int): IO[Int]
    def cityCountryCode(id: Int): IO[String]
}

trait CountryDAO{
    def country(code: String): IO[Country]
    def countryName(code: String): IO[String]
    def countryCapital(code: String): IO[Option[Int]]
}

trait World extends CityDAO with CountryDAO{
    def countries(): IO[List[String]]
    def cities(): IO[List[Int]]
}

defined trait CityDAO
defined trait CountryDAO
defined trait World

In [9]:
val IOListM = Monad[λ[T => IO[List[T]]]]
val IOListF = FunctorFilter[λ[T => IO[List[T]]]]

IOListM: Monad[IO[List[T]]] = ammonite.$sess.common$Helper$$anon$1@7c2063b4
IOListF: FunctorFilter[IO[List[T]]] = ammonite.$sess.common$Helper$$anon$2@5c42f7f3

In [10]:
// No stream fusion
// Can't use for-comprehensions (need a monad for λ[T => IO[List[T]]]) :(
// Needs implicit conversions from IO[T]/IO[Option[T]] to IO[List[T]]

def largeCapitalsIOList(W: World): IO[List[(String, String)]] =
    IOListM.flatMap(W.countries){ countryCode => 
        IOListM.flatMap(
            IOListF.filter(W.country(countryCode))(_.capital.isDefined)
        ){ country => 
            IOListM.map(
                IOListF.filter(W.city(country.capital.get))(_.population > 8000000)
            ){ city => 
                (country.name, city.name)
            }
        }
    }

defined function largeCapitalsIOList

In [11]:
class DoobieWorldIOList(xa: Transactor[IO]) extends World{
    def city(id: Int): IO[City] = 
        sql"select id, name, countryCode, population from city where id = $id"
            .query[City].unique.transact(xa)

    def cityName(id: Int): IO[String] = 
        sql"select name from city where id = $id"
            .query[String].unique.transact(xa)
    
    def cityPopulation(id: Int): IO[Int] = 
        sql"select population from city where id = $id"
            .query[Int].unique.transact(xa)
    
    def cityCountryCode(id: Int): IO[String] = 
        sql"select countryCode from city where id = $id"
            .query[String].unique.transact(xa)

    def country(code: String): IO[Country] = 
        sql"select code, name, capital from country where code = $code"
            .query[Country].unique.transact(xa)
    
    def countryName(code: String): IO[String] =
        sql"select name from country where code = $code"
            .query[String].unique.transact(xa)
    
    def countryCapital(code: String): IO[Option[Int]] =
        sql"select capital from country where code = $code"
            .query[Int].option.transact(xa)
    
    def countries(): IO[List[String]] = 
        sql"select code from country"
            .query[String].to[List].transact(xa)
    
    def cities(): IO[List[Int]] = 
        sql"select id from city"
            .query[Int].to[List].transact(xa)
}

object DoobieWorldIOList extends DoobieWorldIOList(xa)

defined class DoobieWorldIOList
defined object DoobieWorldIOList

In [12]:
largeCapitalsIOList(DoobieWorldIOList).unsafeRunSync.timed

res11: (List[(String, String)], Long) = (
  List(
    ("Indonesia", "Jakarta"),
    ("South Korea", "Seoul"),
    ("Mexico", "Ciudad de M\u00e9xico"),
    ("Russian Federation", "Moscow")
  ),
  5089460430L
)

BIG BIG BIG PROBLEM: no filters AND query avalance AND multiple connections

### with connection pools

In [13]:
val htransactor: Resource[IO, HikariTransactor[IO]] =
    for {
        ce <- ExecutionContexts.fixedThreadPool[IO](32) // our connect EC
        be <- Blocker[IO]    // our blocking EC
        xa <- HikariTransactor.newHikariTransactor[IO](
              "org.postgresql.Driver",                        // driver classname
              "jdbc:postgresql:world",   // connect URL
              "postgres",                                   // username
              "",                                     // password
              ce,                                     // await connection here
              be                                      // execute JDBC operations here
            )
    } yield xa

htransactor: Resource[IO, HikariTransactor[IO]] = Bind(
  Bind(
    Allocate(
      Map(
        Delay(doobie.util.ExecutionContexts$$$Lambda$3476/425990144@39235eea),
        scala.Function1$$Lambda$318/1443055846@7566289f,
        1
      )
    ),
    cats.effect.Resource$$Lambda$3247/609295256@1845ac24
  ),
  ammonite.$sess.cmd12$Helper$$Lambda$3479/846786198@4a1ddea9
)

In [14]:
htransactor.map(xa => new DoobieWorldIOList(xa)).use(largeCapitalsIOList).unsafeRunSync.timed

res13: (List[(String, String)], Long) = (
  List(
    ("Indonesia", "Jakarta"),
    ("South Korea", "Seoul"),
    ("Mexico", "Ciudad de M\u00e9xico"),
    ("Russian Federation", "Moscow")
  ),
  638528506L
)

### with single connection

In [15]:
val DoobieWorldIOListSingle = for {
    con <- Resource.make(
        IO(DriverManager.getConnection("jdbc:postgresql:world", "postgres", "")))(
        connection => IO(connection.close))
    be <- Blocker[IO]
} yield new DoobieWorldIOList(Transactor.fromConnection[IO](con, be))

DoobieWorldIOListSingle: Resource[IO, DoobieWorldIOList] = Bind(
  Allocate(
    Map(
      Delay(ammonite.$sess.cmd14$Helper$$Lambda$3529/1067936711@4175df38),
      scala.Function1$$Lambda$318/1443055846@1ff4c94,
      1
    )
  ),
  ammonite.$sess.cmd14$Helper$$Lambda$3531/2111432197@28f73eef
)

In [16]:
DoobieWorldIOListSingle.use(largeCapitalsIOList).unsafeRunSync.timed

res15: (List[(String, String)], Long) = (
  List(
    ("Indonesia", "Jakarta"),
    ("South Korea", "Seoul"),
    ("Mexico", "Ciudad de M\u00e9xico"),
    ("Russian Federation", "Moscow")
  ),
  632543406L
)

### `Stream[ConnectionIO, ?]`

This API is JDBC-specific. TODO: Stream[IO, ?]

In [17]:
trait CityDAOS{
    def city(id: Int): Stream[ConnectionIO, City]
    def cityName(id: Int): Stream[ConnectionIO, String]
    def cityPopulation(id: Int): Stream[ConnectionIO, Int]
    def cityCountryCode(id: Int): Stream[ConnectionIO, String]
}

trait CountryDAOS{
    def country(code: String): Stream[ConnectionIO, Country]
    def countryName(code: String): Stream[ConnectionIO, String]
    def countryCapital(code: String): Stream[ConnectionIO, Option[Int]]
}

trait WorldS extends CityDAOS with CountryDAOS{
    def countries(): Stream[ConnectionIO, String]
    def cities(): Stream[ConnectionIO, Int]
}

defined trait CityDAOS
defined trait CountryDAOS
defined trait WorldS

In [18]:
object WorldS extends WorldS {
    
    def city(id: Int): Stream[ConnectionIO, City] = 
        sql"select id, name, countryCode, population from city where id = $id"
            .query[City].stream
    
    def cityName(id: Int): Stream[ConnectionIO, String] = 
        sql"select name from city where id = $id"
            .query[String].stream
    
    def cityPopulation(id: Int): Stream[ConnectionIO, Int] = 
        sql"select population from city where id = $id"
            .query[Int].stream

    def cityCountryCode(id: Int): Stream[ConnectionIO, String] = 
        sql"select population from city where id = $id"
            .query[String].stream

    def country(code: String): Stream[ConnectionIO, Country] = 
        sql"select code, name, capital from country where code = $code"
            .query[Country].stream
    
    def countryName(id: String): Stream[ConnectionIO, String] =
        sql"select name from country where id = $id"
            .query[String].stream
    
    def countryCapital(id: String): Stream[ConnectionIO, Option[Int]] =
        sql"select capital from country where id = $id"
            .query[Option[Int]].stream
    
    def countries: Stream[ConnectionIO, String] = 
        sql"select code from country"
            .query[String].stream
    
    def cities: Stream[ConnectionIO, Int] = 
        sql"select code, name, capital from country"
            .query[Int].stream

}

defined object WorldS

In [19]:
def largeCapitalsStream(W: WorldS): Stream[ConnectionIO,(String, String)] = for {
    name <- W.countries
    Country(_, name, Some(capital)) <- W.country(name)
    population <- W.cityPopulation(capital) if population > 8000000
    cName <- W.cityName(capital)
} yield (cName, name)

defined function largeCapitalsStream

In [20]:
largeCapitalsStream(WorldS).compile
    .toList
    .transact(xa)
    .unsafeRunSync
    .timed

res19: (List[(String, String)], Long) = (
  List(
    ("Jakarta", "Indonesia"),
    ("Seoul", "South Korea"),
    ("Ciudad de M\u00e9xico", "Mexico"),
    ("Moscow", "Russian Federation")
  ),
  598482371L
)

BIG PROBLEM: query avalanche (although single connection).